# Part 1 

Let's compute the power level.

In [26]:
def hundred_digit(n):
    nstr = str(n)
    if len(nstr) > 2:
        return int(nstr[-3])
    else:
        raise NotImplementedError('Problem here.')

In [29]:
def compute_power_level(x, y, serial_number):
    rack_id = x + 10
    power_level = rack_id * y
    power_level = power_level + serial_number
    power_level *= rack_id
    power_level = hundred_digit(power_level)
    power_level -= 5
    return power_level

Unit tests:

In [91]:
serial_number = 8

(x, y) = 3, 5

compute_power_level(x, y, serial_number)

4

In [47]:
compute_power_level(122, 79, 57)

-5

In [48]:
compute_power_level(217,196, 39)

0

In [49]:
compute_power_level(101,153,71)

4

Let's also see if we get the two examples on matrices right:

In [37]:
import numpy as np

In [52]:
mat = np.empty((5, 5))

for c in range(32, 37):
    for r in range(44, 49):
        x, y = c, r
        power_level = compute_power_level(x, y, 18)
        mat[r-44, c-32] = power_level

mat

array([[-2., -4.,  4.,  4.,  4.],
       [-4.,  4.,  4.,  4., -5.],
       [ 4.,  3.,  3.,  4., -4.],
       [ 1.,  1.,  2.,  4., -3.],
       [-1.,  0.,  2., -5., -2.]])

In [53]:
mat = np.empty((5, 5))

for c in range(20, 25):
    for r in range(60, 65):
        x, y = c, r
        power_level = compute_power_level(x, y, 42)
        mat[r-60, c-20] = power_level

mat

array([[-3.,  4.,  2.,  2.,  2.],
       [-4.,  4.,  3.,  3.,  4.],
       [-5.,  3.,  3.,  4., -4.],
       [ 4.,  3.,  3.,  4., -3.],
       [ 3.,  3.,  3., -5., -1.]])

Let's build the 300 by 300 grid.

In [54]:
mat = np.empty((300, 300))
for c in range(300):
    for r in range(300):
        x, y = c+1, r+1
        mat[r, c] = compute_power_level(x, y, 5791)

In [55]:
mat

array([[ 3.,  1., -1., ..., -1.,  4., -2.],
       [ 4.,  2.,  1., ..., -2., -2., -1.],
       [-5.,  4.,  2., ..., -3.,  3.,  0.],
       ...,
       [ 2., -1.,  1., ..., -4.,  2., -5.],
       [ 3.,  0.,  3., ...,  4., -3., -4.],
       [-5.,  1.,  4., ...,  3.,  2., -3.]])

Let's find the largest 3x3 region.

In [60]:
def compute_max_3x3(mat):
    rmax, cmax, max_total = None, None, 0
    for c in range(300-3):
        for r in range(300-3):
            region = mat[r:r+3, c:c+3]
            if np.sum(region) > max_total:
                max_total = np.sum(region)
                rmax, cmax = r, c

    return cmax+1, rmax+1

In [61]:
compute_max_3x3(mat)

(20, 68)

# Part 2

We have to rewrite the sliding window code, for which we can also use Numba.

In [76]:
from numba import njit

In [87]:
@njit
def compute_max_any_size(mat, size):
    rmax, cmax, max_total = -2, -2, 0
    for c in np.arange(300-size):
        for r in np.arange(300-size):
            region = mat[r:r+size, c:c+size]
            if np.sum(region) > max_total:
                max_total = np.sum(region)
                rmax, cmax = r, c
    return cmax+1, rmax+1, size, max_total

In [89]:
compute_max_any_size(mat, 3)

(20, 68, 3, 29.0)

In [64]:
from tqdm import tqdm_notebook

In [90]:
scores = []
for size in tqdm_notebook(range(1, 301)):
    scores.append(compute_max_any_size(mat, size))

HBox(children=(IntProgress(value=0, max=300), HTML(value='')))

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/kappamaki/anaconda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/kappamaki/anaconda/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/kappamaki/anaconda/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


In [92]:
max(scores, key=lambda items: items[3])

(231, 273, 16, 111.0)

## Using pythran 

In [93]:
%load_ext pythran.magic

In [98]:
%%pythran
import numpy as np
#pythran export compute_max_any_size_pythran(float[:, :], int)
def compute_max_any_size_pythran(mat, size):
    rmax, cmax, max_total = -2, -2, 0
    for c in np.arange(300-size):
        for r in np.arange(300-size):
            region = mat[r:r+size, c:c+size]
            if np.sum(region) > max_total:
                max_total = np.sum(region)
                rmax, cmax = r, c
    return cmax+1, rmax+1, size, max_total

In [100]:
compute_max_any_size_pythran(mat, 4)

(20, 18, 4, 39.0)

In [101]:
scores = []
for size in tqdm_notebook(range(1, 301)):
    scores.append(compute_max_any_size_pythran(mat, size))

HBox(children=(IntProgress(value=0, max=300), HTML(value='')))

Faster!